In [ ]:
import sys
from pathlib import Path
import pickle
import pandas as pd
import geopandas as gpd

import src.bikewaysim_functions as bikewaysim_functions

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

import impedance_calibration.stochastic_optimization as stochastic_optimization
from network.src import modeling_turns

In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))

In [ ]:
# import test case/ base case (think of a better word) network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
# import the calibration results
with (config['calibration_fp']/"calibration_results.pkl").open('rb') as fh:
    calibration_results = pickle.load(fh)
turn_G = modeling_turns.make_turn_graph(turns)

In [ ]:
#remove imporvements that aren't improvements
links.loc[links['facility_fwd']==links['improvement'],'improvement'] = None
links.loc[links['improvement'].notna(),['facility_fwd','improvement']]

#change cycletracks to multi use paths for now
change_infra = {
    'buffered bike lane': 'bike lane',
    'cycletrack': 'multi use path'
}
links['improvement'] = links['improvement'].apply(lambda x: change_infra.get(x,x))
links['facility_fwd'] = links['facility_fwd'].apply(lambda x: change_infra.get(x,x))

#combine facilities into one
import numpy as np
links['facility_fwd_new'] = np.where(links['improvement'].notna(),links['improvement'],links['facility_fwd'])

In [ ]:
# create the neccessary attributes
links['bike lane'] = (links['facility_fwd'] == 'bike lane').astype(int)
links['multi use path'] = (links['facility_fwd'] == 'multi use path').astype(int)
links.loc[links['multi use path']==1,'lanes'] = 0

In [ ]:
#choose the calibration coefficients to use
calibration_results

In [ ]:
calibration_result = calibration_results[-1]
calibration_result

In [ ]:
betas = list(calibration_result.values())[0:-3]
betas_links = calibration_result['beta_links']
betas_turns = calibration_result['beta_turns']
betas

# Post Match

In [ ]:
with (config['bikewaysim_fp']/"current_results.pkl").open('rb') as fh:
    current_results_dict = pickle.load(fh)

with (config['bikewaysim_fp']/"future_results.pkl").open('rb') as fh:
    future_results_dict = pickle.load(fh)

## Impedance Change

In [ ]:
current_impedance = {key:item['length'] for key, item in current_results_dict.items()}
future_impedance = {key:item['length'] for key, item in future_results_dict.items()}

od_matrix['current_impedance'] = [current_impedance.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]
od_matrix['future_impedance'] = [future_impedance.get(x,np.nan) for x in list((zip(od_matrix['orig_N'],od_matrix['dest_N'])))]

## Betweenness Centrality
We need this to visualize how trips shifted and to detemrine the impedance reduction contribution of each project.

In [ ]:
undirected_btw_centrality = {key:0 for key in links['linkid'].tolist()}
directed_btw_centrality = {key:0 for key in list(zip(links['linkid'],links['reverse_link']))}

In [ ]:
od_matrix

In [ ]:
for od_pair, item in results_dict.items():
    #get number of trips for that od pair
    o_check = od_matrix['orig_N']==od_pair[0]
    d_check = od_matrix['dest_N']==od_pair[1]
    num_trips = od_matrix.loc[o_check & d_check,'trips'].sum()
    
    modeled_edges = item['edge_list']
    modeled_edges = [tuple(x) for x in modeled_edges]
    
    for edge in modeled_edges:
        directed_btw_centrality[edge] += num_trips
    
    modeled_edges = item['edge_list']
    modeled_edges = [x[0] for x in modeled_edges]

    for edge in modeled_edges:
        undirected_btw_centrality[edge] += num_trips

In [ ]:
#filter
undirected_btw_centrality = {key:item for key,item in undirected_btw_centrality.items() if item > 0}
directed_btw_centrality = {key:item for key,item in directed_btw_centrality.items() if item > 0}

In [ ]:
test = links[links['reverse_link']==False].copy()
test['counts'] = test['linkid'].map(undirected_btw_centrality)
test.plot('counts')

In [ ]:
test = links.copy()#[links['reverse_link']==False].copy()
test['counts'] = test.apply(lambda x: directed_btw_centrality.get((x['linkid'],x['reverse_link'])),axis=1)

In [ ]:
test['impedance_change'] = current_links['link_cost'] - future_links['link_cost']

In [ ]:
#should be change in btw centrality? assuming that 

In [ ]:
test['impedance_change_cont'] = test['counts'] * test['impedance_change']

In [ ]:
#import the coa id back in so that we can aggregate by the faciility
facility_ids = gpd.read_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')[['coa_id','linkid']]

In [ ]:
test = pd.merge(test,facility_ids,on='linkid',how='left')

In [ ]:
result = test[test['impedance_change_cont']>0].groupby(['coa_id','improvement'])['impedance_change_cont'].sum().sort_values(ascending=False)
result.reset_index(inplace=True)

In [ ]:
facility_ids = gpd.read_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')
facility_ids = facility_ids.dissolve('coa_id').reset_index()

In [ ]:
facility_ids = pd.merge(facility_ids,result,on='coa_id')
facility_ids.explore('impedance_change_cont')

In [ ]:
results_dict

In [ ]:
#od_matrix['current_impedance'] = 
results_dict.apply(lambda x: current_results_dict.get((x['origin_N'],),axis=1)

In [ ]:
# find the impedance for each trip

In [ ]:
results_dict[(6846937343, 5416049647)].keys()

In [ ]:
#figure out why this is na
pd.Series([item['length'] for key, item in results_dict.items()]).isna().sum()

In [ ]:
pd.Series([item['length'] for key, item in results_dict.items()]).isna().sum()

In [ ]:
len(results_dict)

In [ ]:
# get the results and assigned to ods (store edge sequence in a dict soemwehre)
ods.map(results_dict)

In [ ]:
# calculate btwn centrality



In [ ]:
# import test case/ base case (think of a better word) network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
# import the calibration results
with (config['calibration_fp']/"calibration_results.pkl").open('rb') as fh:
    calibration_results = pickle.load(fh)
turn_G = modeling_turns.make_turn_graph(turns)